## Face Recognition
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습과 추론을 진행.
    3. Extra
        - 조원들의 얼굴 이미지를 수집한 후, 조원들을 구분하는 모델을 만듦.

### Google Drive 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터셋 압축 해제
- Face_Recognition.zip : 이미지 데이터 압축 파일

In [2]:
import os, glob, shutil, zipfile
from tqdm import tqdm

In [3]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[-20:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[-20:-4]}/')

In [11]:
dataset_path = '/content/drive/MyDrive/미프5차_YOLO/Face_Recognition.zip'

In [12]:
dataset_extract(dataset_path)

Extracting: 100%|██████████| 11960/11960 [00:11<00:00, 1021.10files/s]


### 데이터셋 살펴보기

In [13]:
print(os.listdir('/content/Face_Recognition/'))
print(os.listdir('/content/Face_Recognition/train'))
print(os.listdir('/content/Face_Recognition/valid'))

['valid', 'train']
['IMG_5587_JPG_jpg.rf.5c87d75cc1126ee2f2a5eb2008bccdf0.txt', 'david-beckham-lede-5--1-_jpg.rf.63f1cc63bb71e965a8706b94200bb854.jpg', 'fade009d75726082d817f3984c38a6094a-biden-rsquare-w1200_jpg.rf.3fd9464aa74207df4c6c574d217e8c76.jpg', '_117080140_4a42d5ba-d752-4926-a46f-058d0a3987a4_jpg.rf.69f38ea4eeaf2ac110f19a9de02c823c.jpg', 'default_jpg.rf.1802874969d99b1e2c298a4c1f0c57e9.jpg', 'IMG_5667_JPG_jpg.rf.eb2ab0e8552d11492f42c5eb4c79b566.jpg', 'shutterstock_editorial_10434333bm---Copy---Copy---Copy---Copy_jpg.rf.7a8bd4c8ae35185905aae98db6ec10e9.jpg', 'Donald_Trump_March_2015--1-_jpg.rf.c00995447b8f68d4c1d5bfc9cd45a651.jpg', 'f1YV7XUW_png_jpg.rf.4f8fd0f66e3139e378759912b478390c.txt', 'IMG_5526_JPG_jpg.rf.f80e348d66414d5aa124e28bbc529e77.jpg', '788px-President_Barack_Obama-_2012_portrait_crop--1----Copy---Copy---Copy---Copy_jpg.rf.e0d49ffccd57f7fb21139b27f09896f8.jpg', 'images--17----Copy---Copy---Copy---Copy_jpg.rf.7d1ef06e80af1cf6782c3d2371bf9785.jpg', 'Cunningham-Obama

In [14]:
print(sorted(glob.glob('/content/Face_Recognition/train/*.jpg')))
print(sorted(glob.glob('/content/Face_Recognition/train/*.txt')))

['/content/Face_Recognition/train/-----------------------------------------------------------------------------------------26bdffa9-f496-4ec8-8f9c-0801492f406a_jpg.rf.10f1cb145a8c687521fc909f59db3380.jpg', '/content/Face_Recognition/train/-----------------------------------------------------------------------------------------26bdffa9-f496-4ec8-8f9c-0801492f406a_jpg.rf.4158f33f5c4dabe06a7e71b613e97574.jpg', '/content/Face_Recognition/train/-----------------------------------------------------------------------------------------26bdffa9-f496-4ec8-8f9c-0801492f406a_jpg.rf.ad183635ffe582637302b23c9eabb474.jpg', '/content/Face_Recognition/train/--------------------------------------------------------_jpg.rf.0b5f31f3f7e5aa5eafadd3a6f7448727.jpg', '/content/Face_Recognition/train/--------------------------------------------------------_jpg.rf.1fee11a3fd9beb8a2ef4d679bcc807c9.jpg', '/content/Face_Recognition/train/--------------------------------------------------------_jpg.rf.6930a7663809980

In [15]:
def count_file(dir):
  files = os.listdir(dir)
  jpg_count = 0
  txt_count = 0

  for file in files:
    if file.endswith('.jpg'):
      jpg_count += 1
    elif file.endswith('.txt'):
      txt_count += 1
  print(f'Jpg 개수: {jpg_count}')
  print(f'Text 개수: {txt_count}')
  return

In [16]:
train_dir = '/content/Face_Recognition/train/'
valid_dir = '/content/Face_Recognition/valid/'

# train 폴더내 파일 개수
count_file(train_dir)

# valid 폴더내 파일 개수
count_file(valid_dir)

Jpg 개수: 5214
Text 개수: 5214
Jpg 개수: 765
Text 개수: 765


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식.
- 해당 형식에 맞춰 폴더를 만듦.

In [17]:
def train_find(dir):
  # jpg, txt 찾음
  images = glob.glob(os.path.join(dir, '*.jpg'))
  texts = glob.glob(os.path.join(dir, '*.txt'))
  # 디렉토리 경로 설정
  train_img_dir = '/content/Face_Recognition/train/images'
  train_label_dir = '/content/Face_Recognition/train/labels'

  # 디렉토리 없으니 생성
  os.makedirs(train_img_dir, exist_ok=True)
  os.makedirs(train_label_dir, exist_ok=True)

  for img in images:
    if os.path.exists(img): # 이미지 파일 존재 확인
      shutil.move(img, train_img_dir)
  for txt in texts:
    if os.path.exists(txt): # 텍스트 파일 존재 확인
      shutil.move(txt, train_label_dir)
  print(f'모든 파일 이동')

  return


def valid_find(dir):
  # jpg, txt 찾음
  images = glob.glob(os.path.join(dir, '*.jpg'))
  texts = glob.glob(os.path.join(dir, '*.txt'))
  # 디렉토리 경로 설정
  valid_img_dir = '/content/Face_Recognition/valid/images'
  valid_label_dir = '/content/Face_Recognition/valid/labels'

  # 디렉토리 없으니 생성
  os.makedirs(valid_img_dir, exist_ok=True)
  os.makedirs(valid_label_dir, exist_ok=True)

  for img in images:
    if os.path.exists(img): # 이미지 파일 존재 확인
      shutil.move(img, valid_img_dir)
  for txt in texts:
    if os.path.exists(txt): # 텍스트 파일 존재 확인
      shutil.move(txt, valid_label_dir)
  print(f'모든 파일 이동')

  return

### 폴더에 맞게 파일 이동
- 이전 단계에서 만든 하위 폴더에 이미지 파일과 텍스트 파일을 이동.

In [18]:
train_find(train_dir)
valid_find(valid_dir)

모든 파일 이동
모든 파일 이동


### YOLO 모델에 적용할 YAML 생성하기
- 클래스는 총 13개. 클래스 순서에 주의.
- 'Barack Obama', 'Che Guevara', 'Cristiano Ronaldo', 'David Beckham', 'Donald Trump', 'Elon Musk', 'Joe Biden', 'Lionel Messi', 'Mark Zuckerberg', 'Oprah Winfrey', 'Steve Harvey', 'Steve Jobs', 'Zaha Hadid'

In [19]:
import yaml

data = {
    'train': './train/images/',
    'val': './valid/images/',
    'nc': 13,
    'names': [
        'Barack Obama',
        'Che Guevara',
        'Cristiano Ronaldo',
        'David Beckham',
        'Donald Trump',
        'Elon Musk',
        'Joe Biden',
        'Lionel Messi',
        'Mark Zuckerberg',
        'Oprah Winfrey',
        'Steve Harvey',
        'Steve Jobs',
        'Zaha Hadid'
    ]
}

# yaml 파일
with open('./Face_Recognition/data_info.yaml', 'w') as yaml_file:
  yaml.dump(data, yaml_file, default_flow_style=False)

### YOLO v8 모델 사용
- yaml 파일의 경로 설정에 주의.

In [20]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [21]:
from ultralytics import YOLO, settings

settings['datasets_dir'] = '/content/'

In [22]:
model = YOLO(model='yolov8s.pt', task='detect')

100%|██████████| 21.5M/21.5M [00:00<00:00, 155MB/s]


In [ ]:
model.train(model='/content/yolov8s.pt',
            data='/content/Face_Recognition/data_info.yaml',
            epochs=250,
            patience=50,
            pretrained=True,
            verbose=True,
            seed=777)

Ultralytics YOLOv8.2.4 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train3/weights/best.pt, data=/content/data_info.yaml, epochs=220, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train34, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=777, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

train: Scanning /content/train/labels.cache... 5214 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5214/5214 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/valid/labels.cache... 765 images, 0 backgrounds, 0 corrupt: 100%|██████████| 765/765 [00:00<?, ?it/s]


Plotting labels to runs/detect/train34/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train34
Starting training for 220 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        765        765      0.753      0.587      0.645      0.486
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 37, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



52 epochs completed in 0.521 hours.
Optimizer stripped from runs/detect/train34/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train34/weights/best.pt, 22.5MB

Validating runs/detect/train34/weights/best.pt...
Ultralytics YOLOv8.2.4 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 11130615 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:04<00:00,  5.07it/s]


                   all        765        765      0.912      0.639      0.769      0.678
          Barack Obama        765         56      0.932      0.732      0.837       0.75
           Che Guevara        765         38       0.81      0.674      0.778       0.71
     Cristiano Ronaldo        765         52      0.943      0.632      0.815      0.775
         David Beckham        765         75      0.807      0.827      0.852       0.73
          Donald Trump        765         74      0.953      0.351      0.609      0.544
             Elon Musk        765         74      0.959      0.757      0.882      0.798
             Joe Biden        765         80      0.875      0.725      0.796      0.704
          Lionel Messi        765         69      0.915      0.899       0.93      0.843
       Mark Zuckerberg        765         39      0.932      0.462      0.597      0.498
         Oprah Winfrey        765         49      0.924      0.739      0.841      0.726
          Steve Harve

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x781638343310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.04504

In [ ]:
result = model.predict(source='/content/Face_Recognition/valid/images',
        #      conf=0.05,
        #      iou=0.2,
              save=True, save_txt=True)


image 1/765 /content/valid/images/--------------------------------------------------------_jpg.rf.f7140876c651456e7478ed60c5d79f53.jpg: 640x640 1 Lionel Messi, 12.7ms
image 2/765 /content/valid/images/------------------------------------------------_jpeg_jpg.rf.a5c654226ab7a5870f01f3481718b832.jpg: 640x640 1 Donald Trump, 9.9ms
image 3/765 /content/valid/images/------------------------------------------------_jpeg_jpg.rf.e995e0fdc7313eafd4624e271d9afc13.jpg: 640x640 1 Elon Musk, 1 Lionel Messi, 1 Mark Zuckerberg, 9.9ms
image 4/765 /content/valid/images/----------------------------_png_jpg.rf.69cc0d78e68d34bc3b66a79b4dd0d2f3.jpg: 640x640 2 Elon Musks, 1 Lionel Messi, 2 Zaha Hadids, 9.7ms
image 5/765 /content/valid/images/----------------------------_png_jpg.rf.b40a3a9b61b7cc510a17e49639076fb7.jpg: 640x640 2 Zaha Hadids, 9.6ms
image 6/765 /content/valid/images/-------------------------_jpg.rf.7c338e573759567c0f987ee097f4db89.jpg: 640x640 1 Barack Obama, 1 Donald Trump, 1 Elon Musk, 1 Ma

## Extra) 조원들의 얼굴을 구분하는지 테스트!
1. 데이터 수집 : 조원들의 얼굴 사진을 수집한다! -> 이미지 파일
2. Annotation 작업 : 얼굴 부위에 대해 라벨링을 한다! -> txt 파일
3. 본인 구글 드라이브에 업로드 및 Colab과의 연동!
4. YAML 파일 생성 : 조원들의 사진이 있는 경로와 클래스 수, 클래스 이름 정보가 있는 YAML을 새로 생성!
5. 추가 학습 : 위의 단계에서 생성된 최적의 가중치를 가진 모델을 불러와 추가 학습을 시킨다!
6. 테스트 : 조원들을 잘 인식하는지 predict 해본다!

In [23]:
dataset_path = '/content/drive/MyDrive/미프5차_YOLO/YOLO_prac_28.zip'

In [ ]:
dataset_extract(dataset_path)

### 데이터셋 살펴보기

In [ ]:
print(os.listdir('/content/OLO/YOLO_prac_28/YOLO_prac_28'))

In [ ]:
print(sorted(glob.glob('/content/OLO/YOLO_prac_28/YOLO_prac_28/train/images/*.jpg')))
print(sorted(glob.glob('/content/OLO/YOLO_prac_28/YOLO_prac_28/train/labels/*.txt')))

In [8]:
def count_jpg_file(dir):
  files = os.listdir(dir)
  jpg_count = 0
  txt_count = 0

  for file in files:
    if file.endswith('.jpg'):
      jpg_count += 1
  print(f'Jpg 개수: {jpg_count}')
  return

def count_txt_file(dir):
  files = os.listdir(dir)
  jpg_count = 0
  txt_count = 0

  for file in files:
    if file.endswith('.txt'):
      txt_count += 1
  print(f'Text 개수: {txt_count}')
  return

In [ ]:
train_jpg_dir = '/content/OLO/YOLO_prac_28/YOLO_prac_28/train/images/'
valid_jpg_dir = '/content/OLO/YOLO_prac_28/YOLO_prac_28/valid/images/'

train_txt_dir = '/content/OLO/YOLO_prac_28/YOLO_prac_28/train/labels/'
valid_txt_dir = '/content/OLO/YOLO_prac_28/YOLO_prac_28/valid/labels/'

# train 폴더내 파일 개수
count_jpg_file(train_jpg_dir)
count_txt_file(train_txt_dir)
# valid 폴더내 파일 개수
count_jpg_file(valid_jpg_dir)
count_txt_file(valid_txt_dir)

### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식에 따라 만듦.

In [ ]:
def train_jpg_find(dir):
  # jpg, txt 찾음
  images = glob.glob(os.path.join(dir, '*.jpg'))
  # 디렉토리 경로 설정
  train_img_dir = '/content/YOLO_prac_28/train/images'

  # 디렉토리 없으니 생성
  os.makedirs(train_img_dir, exist_ok=True)
  for img in images:
    if os.path.exists(img): # 이미지 파일 존재 확인
      shutil.move(img, train_img_dir)
  print(f'모든 파일 이동')
  return

def train_txt_find(dir):
  # jpg, txt 찾음
  texts = glob.glob(os.path.join(dir, '*.txt'))
  # 디렉토리 경로 설정
  train_label_dir = '/content//YOLO_prac_28/train/labels'

  # 디렉토리 없으니 생성
  os.makedirs(train_label_dir, exist_ok=True)

  for txt in texts:
    if os.path.exists(txt): # 텍스트 파일 존재 확인
      shutil.move(txt, train_label_dir)
  print(f'모든 파일 이동')

  return

def valid_jpg_find(dir):
  # jpg, txt 찾음
  images = glob.glob(os.path.join(dir, '*.jpg'))
  # 디렉토리 경로 설정
  valid_img_dir = '/content/YOLO_prac_28/valid/images'

  # 디렉토리 없으니 생성
  os.makedirs(valid_img_dir, exist_ok=True)

  for img in images:
    if os.path.exists(img): # 이미지 파일 존재 확인
      shutil.move(img, valid_img_dir)
  print(f'모든 파일 이동')

  return


def valid_txt_find(dir):
  # jpg, txt 찾음
  texts = glob.glob(os.path.join(dir, '*.txt'))
  # 디렉토리 경로 설정
  valid_label_dir = '/content/YOLO_prac_28/valid/labels'

  # 디렉토리 없으니 생성
  os.makedirs(valid_label_dir, exist_ok=True)

  for txt in texts:
    if os.path.exists(txt): # 텍스트 파일 존재 확인
      shutil.move(txt, valid_label_dir)
  print(f'모든 파일 이동')

  return

### 폴더에 맞게 파일 이동
- 이전 단계에서 만든 하위 폴더에 이미지 파일과 텍스트 파일을 이동.

In [ ]:
train_jpg_find(train_jpg_dir)
train_txt_find(train_txt_dir)
valid_jpg_find(valid_jpg_dir)
valid_txt_find(valid_txt_dir)

모든 파일 이동
모든 파일 이동
모든 파일 이동
모든 파일 이동


### YAML 새로 생성하기
- 클래스의 수는 조원의 수
- Annotation 작업 과정에서 정해진 클래스의 순서 규칙 준수

In [ ]:
import yaml

data = {
    'train': './train/images/',
    'val': './valid/images/',
    'nc': 7,
    'names': [
        'Kim_Young_bin',
        'Seo_Hyun_bin',
        'Park_Seong_ah',
        'Bak_Hyeon_guen',
        'Choi_Gyu_jin',
        'Huh_Seung_bin',
        'Hwang_Jong_won'
    ]
}

# yaml 파일
with open('./YOLO_prac_28/data_info2.yaml', 'w') as yaml_file:
  yaml.dump(data, yaml_file, default_flow_style=False)

### 모델 추가 학습하기

In [ ]:
model = YOLO(model='/content/YOLO_prac_28/best.pt', task='detect')

In [ ]:
model.train(model='/content/YOLO_prac_28/best.pt',
            data='/content/YOLO_prac_28/data_info2.yaml',
            epochs=250,
            patience=50,
            pretrained=True,
            verbose=True,
            seed=777)

Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/미프5차_YOLO/YOLO_prac_28/best.pt, data=/content/drive/MyDrive/미프5차_YOLO/YOLO_prac_28/data_info.yaml, epochs=5, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=777, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=F

100%|██████████| 755k/755k [00:00<00:00, 37.7MB/s]


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

train: Scanning /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/train/labels... 663 images, 0 backgrounds, 0 corrupt: 100%|██████████| 663/663 [01:46<00:00,  6.24it/s]

train: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/train/images/20240430_111039_001_saved (1).jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/train/images/20240430_111039_001_saved.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/train/images/20240430_111039_002_saved (1).jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/train/images/20240430_111039_002_saved.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/train/images/20240430_111039_003_saved (1).jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmP

...

val: Scanning /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/valid/labels... 225 images, 30 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:33<00:00,  7.61it/s]

val: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/valid/images/20240430_111055_007_saved (1).jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/valid/images/20240430_111055_007_saved.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/valid/images/20240430_111055_008_saved (1).jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/valid/images/20240430_111055_008_saved.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/valid/images/20240430_111055_009_saved (1).jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac

val: New cache created: /content/drive/.shortcut-targets-by-id/15zgorgGiRs9n_sL7gvdaHLd0udDEmPJ9/YOLO_prac_28/valid/labels.cache
Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5         0G      1.704      6.381      1.946         33        640:   7%|▋         | 3/42 [01:13<15:55, 24.50s/it]


KeyboardInterrupt: 

In [ ]:
result = model.predict(source='/content/YOLO_prac_28/valid/images',
              save=True, save_txt=True)


image 1/765 /content/valid/images/--------------------------------------------------------_jpg.rf.f7140876c651456e7478ed60c5d79f53.jpg: 640x640 1 Lionel Messi, 12.7ms
image 2/765 /content/valid/images/------------------------------------------------_jpeg_jpg.rf.a5c654226ab7a5870f01f3481718b832.jpg: 640x640 1 Donald Trump, 9.9ms
image 3/765 /content/valid/images/------------------------------------------------_jpeg_jpg.rf.e995e0fdc7313eafd4624e271d9afc13.jpg: 640x640 1 Elon Musk, 1 Lionel Messi, 1 Mark Zuckerberg, 9.9ms
image 4/765 /content/valid/images/----------------------------_png_jpg.rf.69cc0d78e68d34bc3b66a79b4dd0d2f3.jpg: 640x640 2 Elon Musks, 1 Lionel Messi, 2 Zaha Hadids, 9.7ms
image 5/765 /content/valid/images/----------------------------_png_jpg.rf.b40a3a9b61b7cc510a17e49639076fb7.jpg: 640x640 2 Zaha Hadids, 9.6ms
image 6/765 /content/valid/images/-------------------------_jpg.rf.7c338e573759567c0f987ee097f4db89.jpg: 640x640 1 Barack Obama, 1 Donald Trump, 1 Elon Musk, 1 Ma

# YOLO 조원 얼굴로만 학습 및 탐지

In [ ]:
model = YOLO(model='yolov8s.pt', task='detect')

In [ ]:
model.train(model='/content/yolov8s.pt',
            data='/content/YOLO_prac_28/data_info2.yaml',
            epochs=250,
            patience=50,
            pretrained=True,
            verbose=True,
            seed=777)

In [ ]:
result = model.predict(source='/content/YOLO_prac_28/valid/images',
              save=True, save_txt=True)